In [ ]:
''' This code copies both required yml files into the analysis folders
FROM:
    C:\git\Olfactometer-cal\data\Olfactometer.harp\device.yml
    C:\git\Olfactometer-cal\data\AnalogInput.harp\device.yml

TO:
    C:\Data\Olfact\{folder}\{subfolder}\behavior\Olfactometer.harp\device.yml
    C:\Data\Olfact\{folder}\{subfolder}\behavior\AnalogInput.harp\device.yml

 '''

import os
import shutil

# source files
SRC_OLFA = r"C:\git\Olfactometer-cal\data\Olfactometer.harp\device.yml"
SRC_ANALOG = r"C:\git\Olfactometer-cal\data\AnalogInput.harp\device.yml"

if __name__ == "__main__":
    base_root = r"C:\Data\Olfact"

    for rig_folder in os.listdir(base_root):
        rig_path = os.path.join(base_root, rig_folder)
        if not os.path.isdir(rig_path):
            continue

        for subfolder in os.listdir(rig_path):
            subfolder_path = os.path.join(rig_path, subfolder)
            if not os.path.isdir(subfolder_path):
                continue

            print("Preparing test folder:", subfolder_path)

            # -----------------------------------------------------
            # 1️⃣ Create target behavior subfolders
            # -----------------------------------------------------
            behavior_path = os.path.join(subfolder_path, "behavior")

            olfa_target = os.path.join(behavior_path, "Olfactometer.harp")
            analog_target = os.path.join(behavior_path, "AnalogInput.harp")

            os.makedirs(olfa_target, exist_ok=True)
            os.makedirs(analog_target, exist_ok=True)

            # -----------------------------------------------------
            # 2️⃣ Copy the files
            # -----------------------------------------------------
            shutil.copy(SRC_OLFA, os.path.join(olfa_target, "device.yml"))
            shutil.copy(SRC_ANALOG, os.path.join(analog_target, "device.yml"))

            print("  → Copied behavior files")



Preparing test folder: C:\Data\Olfact\ephys_5\ephys_5_2025-10-20T214318Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\OIfactometer-5A\dipropilsulfide_pentyl_acetate_hexanal-1percent_2025-11-07T064350Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\Olfactometer-13A\olfactometer_13A_2025-11-06T052526Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\Olfactometer-13A\olfactometer_13A_2025-11-06T062858Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\olfactometer-4A\olfactometer_4A_2025-11-12T052044Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\olfactometer-4B\olfactometer_4B_2025-11-12T052057Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\olfactometer-4C\olfactometer-4C_2025-11-07T070059Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\olfactometer-4C\olfactometer_4C_2025-11-12T051921Z
  → Copied behavior files
Preparing test folder: C:\Data\Olfact\olfactometer-4C\olfact

In [2]:
### OLFACTOMETER REGISTERS LIST ###
import harp  
# Create reader for olfactometer data
OlfReader = harp.create_reader(PATH + "/behavior/Olfactometer.harp", include_common_registers=False)

print(list(OlfReader.registers.keys())) # List of all olfactometer registers

['WhoAmI', 'HardwareVersionHigh', 'HardwareVersionLow', 'AssemblyVersion', 'CoreVersionHigh', 'CoreVersionLow', 'FirmwareVersionHigh', 'FirmwareVersionLow', 'TimestampSeconds', 'TimestampMicroseconds', 'OperationControl', 'ResetDevice', 'DeviceName', 'SerialNumber', 'ClockConfiguration', 'EnableFlow', 'Flowmeter', 'DI0State', 'Channel0UserCalibration', 'Channel1UserCalibration', 'Channel2UserCalibration', 'Channel3UserCalibration', 'Channel4UserCalibration', 'Channel3UserCalibrationAux', 'UserCalibrationEnable', 'Channel0TargetFlow', 'Channel1TargetFlow', 'Channel2TargetFlow', 'Channel3TargetFlow', 'Channel4TargetFlow', 'ChannelsTargetFlow', 'Channel0ActualFlow', 'Channel1ActualFlow', 'Channel2ActualFlow', 'Channel3ActualFlow', 'Channel4ActualFlow', 'Channel0Frequency', 'Channel1Frequency', 'Channel2Frequency', 'Channel3Frequency', 'Channel4Frequency', 'Channel0DutyCycle', 'Channel1DutyCycle', 'Channel2DutyCycle', 'Channel3DutyCycle', 'Channel4DutyCycle', 'DigitalOutputSet', 'DigitalOu

In [ ]:
### OLFACTOMETER - ALL DATA TO CSV ###
import harp 
import pandas as pd

# Create reader for olfactometer data
OlfReader = harp.create_reader(PATH + "/behavior/Olfactometer.harp", include_common_registers=False)

# Containers
metadata_rows = {}
alldf = pd.DataFrame()

# Loop through all olfactometer registers
for name, reg in OlfReader.registers.items():
    try:
        df = reg.read()
        df.index = df.index.round(3)

        # Rename columns only if they contain "Channel" Eg: ['Flowmeter', 'Channel1', 'Channel2'] will be ['Flowmeter', 'Flowmeter-Ch1', 'Flowmeter-Ch2']
        new_cols = []
        for col in df.columns:
            if "channel" in col.lower():
                new_cols.append(f"{name}-Ch{col[7:]}")
            else:
                new_cols.append(col)
        df.columns = new_cols

        # If the register has less than 5 rows, treat it as metadata
        if len(df) < 5:
            metadata_rows[name] = df.iloc[0].to_dict()
        else:
            alldf = pd.concat([alldf, df], axis=1)


    except Exception as e:
        print(f"Skipping register {name} {df.head()}: {e}")

# Create metadata DataFrame
if metadata_rows:
    metadata_df = pd.DataFrame.from_dict(metadata_rows, orient='index')
    print("\n📋 Metadata DataFrame:")
    print(metadata_df)
else:
    print("⚠️ No metadata rows found.")

alldf.index = (alldf.index - alldf.index[0])
# Save merged DataFrame
alldf.to_csv('merged_df.csv', index=True)


Skipping register EndValveState              EndValve0  EndValve1  ValveDummy
Time                                         
4693311.086      False      False       False
4693311.123      False      False       False
4693321.130       True      False       False
4693323.135      False      False       False
4693328.147       True      False       False: Reindexing only valid with uniquely valued Index objects

📋 Metadata DataFrame:
                              WhoAmI  HardwareVersionHigh  HardwareVersionLow  \
WhoAmI                        1140.0                  NaN                 NaN   
HardwareVersionHigh              NaN                  1.0                 NaN   
HardwareVersionLow               NaN                  NaN                 1.0   
AssemblyVersion                  NaN                  NaN                 NaN   
CoreVersionHigh                  NaN                  NaN                 NaN   
...                              ...                  ...                 ...  